# Room Detection AI

Building an AI using GANs to automatically detect which room in the house a picture is of. So far I have compiled a dataset of over 27000 photos across nine classes:

- Bedroom
- Bathroom
- Kitchen
- Study
- Entrance Hall
- Utility Room
- Dinning Room
- Living Room
- Conservatory

I am writing the code initially in this Juptyer Notebook, before training in a Google Colab.

## Import libraries

In [1]:
import pandas as pd
import numpy as np
import cv2 # may not use
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import os
from os import listdir
import tensorflow as tf



## import the data

In [2]:
# to do import data 
# split into training and validation
# augment data
# check weights of each class to make sure each are equal. 
# initiate transfer learning 
# build a new classification head
# compile the model 
# run a baseline training


In [3]:
path = "/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/"
CATEGORIES = ['bathroom', 'bedroom', 'kitchen', 'home_study', 'entrance_hall', 'utility_room', 'conservatory', 'dining_room', 'living_room']

In [4]:
from PIL import Image
import glob
image_list = []
for filename in glob.glob(path +'*.jpg'):
    im=Image.open(filename)
    image_list.append(im)

In [5]:
for name in os.listdir(path):
    print(name)

bedroom
conservatory
.DS_Store
bathroom
dining_room
living_room
home_study
entrance_hall
kitchen
utility_room


In [6]:
pwd

'/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection'

In [7]:


import cv2
import os

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename, ))
        if img is not None:
            images.append(img)
    return images
folder=path

In [8]:
load_images_from_folder(folder)

[]

In [9]:
# load each subfolder seperately - convert to array and then concatonate the data into one dataframe. 

In [10]:
# file paths
root_path = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/'

bedroom_path = root_path + 'bedroom'
conservatory_path = root_path + 'conservatory'
bathroom_path = root_path + 'bathroom'
dining_path = root_path + 'dining_room'
living_path = root_path + 'living_room'
study_path = root_path + 'home_study'
entrance_path = root_path + 'entrance_hall'
kitch_path = root_path + 'kitchen'
util_path = root_path + 'utility_room'

In [15]:
import os
import shutil

# add all images to a new folder named all_images

train_dir = root_path
dest_dir = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/all_images'
counter = 0

for subdir, dirs, files in os.walk(train_dir):
    #print(files)
    for file in files:
        full_path = os.path.join(subdir, file)
        shutil.copy(full_path, dest_dir)
        counter += 1
print(counter)

27677


In [12]:
# store each datapoint name as one array

subdirs, dirs, files = os.walk(dest_dir).__next__()
m = len(files)
print(m)

filenames = []
labels = np.zeros((m, 1))

images_dir = dest_dir
filenames_counter = 0
labels_counter = -1

for subdir, dirs, files in os.walk(train_dir):
    for file in files:
        filenames.append(file)
        filenames_counter += 1
        # print(labels[filenames_counter, 0])
        #labels[filenames_counter, 0] = labels_counter
        
    labels_counter += 1
    
    
print(len(filenames))
print(labels.shape)

27657
27677
(27657, 1)


In [13]:
# saving the filename array as .npy file
np.save('filenames.npy', filenames)

import keras
from keras.utils import to_categorical

# One hot vector representation of labels
y_labels_one_hot = to_categorical(labels)

# saving the y_labels_one_hot array as a .npy file
np.save('y_labels_one_hot.npy', y_labels_one_hot)

In [14]:
# shuffle the dataset

from sklearn.utils import shuffle

filenames_shuffled, y_labels_one_hot_shuffled = shuffle(filenames, y_labels_one_hot)

# saving the shuffled file.
# you can load them later using np.load().
np.save('y_labels_one_hot_shuffled.npy', y_labels_one_hot_shuffled)
np.save('filenames_shuffled.npy', filenames_shuffled)

ValueError: Found input variables with inconsistent numbers of samples: [27677, 27657]

## Import the base model

In [ ]:
# Image size, mobile net uses 244 rather than 256
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = IMG_WIDTH, IMG_HEIGHT

# instantiate the MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
image_batch, label_batch = next(iter(train_dataset)) # the next iteration in the dataset, so the first image
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# freeze the convolutional base
base_model.trainable=False

In [ ]:
# add classification head

In [ ]:
# convert the features to a single 1280-element vector per image
global_av_layer = tf.keras.layers.GlobalAveragePooling2D() # averages over a 5x5 spatial 
feature_batch_av = global_av_layer(feature_batch)
print(feature_batch_av.shape)

In [ ]:
# apply a dense layer to convert these features into a single prediction per image
# no activation needed as the prediction will be treated as a logit (mapping of probabilities to Real Numbers)

pred_layer = tf.keras.layers.Dense(1)
pred_batch = pred_layer(feature_batch_av)
pred_batch.shape

## Build the model

In [ ]:
# augmentate the data 

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

# rescale the pixel values to match the expected values of the MobileNetV2 model

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# chain together data augmentation, rescaling, base_model and feature extractor layers useing the Keras Functional API

inputs = tf.keras.Input(shape=(160,160,3)) # image size and channels
# data augmentation layer
x = data_aug(inputs)
# preprocess, feed x into and reassign variable
x = preprocess_input(x)
# basemodel, set training =False for the BN layer
x = base_model(x, training=False)
# feature extraction
x = global_av_layer(x)
# add a dropout layer 
x = tf.keras.layers.Dropout(0.2)(x)

outputs = pred_layer(x)
model = tf.keras.Model(inputs, outputs)

## Compile the model

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

In [ ]:
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

## Fine Tune and improve the model